<a href="https://colab.research.google.com/github/emmanuelmokel/PPL-practice/blob/main/NestedRHat_TFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!rm -Rf probability
!rm -Rf fun_mc
!rm -Rf inference_gym
!git clone https://github.com/tensorflow/probability.git
!mv probability/spinoffs/fun_mc/fun_mc .
!mv probability/spinoffs/inference_gym/inference_gym .
!pip install tf-nightly tfp-nightly jax jaxlib

!pip install immutabledict

Cloning into 'probability'...
remote: Enumerating objects: 100058, done.
remote: Counting objects: 100% (4884/4884), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 100058 (delta 4769), reused 4738 (delta 4711), pack-reused 95174
Receiving objects: 100% (100058/100058), 136.95 MiB | 24.38 MiB/s, done.
Resolving deltas: 100% (81891/81891), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      S

In [4]:
import numpy as np
from matplotlib.pyplot import *

import jax
from jax import random
from jax import numpy as jnp


#!pip install tfp-nightly inference_gym
from inference_gym import using_jax as gym

#from tensorflow_probability.spinoffs import using_jax as fun_mcmc
from fun_mc import using_jax as fun_mcmc

from tensorflow_probability.python.internal import prefer_static as ps
from tensorflow_probability.python.internal import unnest

import tensorflow_probability as _tfp
tfp = _tfp.substrates.jax
tfd = tfp.distributions
tfb = tfp.bijectors

tfp_np = _tfp.substrates.numpy
tfd_np = tfp_np.distributions

In [5]:
# Working with the 'Banana' Posterior Target Density

target = gym.targets.VectorModel(gym.targets.Banana(),
                                 flatten_sample_transformations = True)
num_dimensions = target.event_shape[0]
# Is the heuristic for the initial value of \epsilon given in the paper used in practice?
init_step_size = 1.

def target_log_prob_fn(x):
   y = target.default_event_space_bijector(x)
   fldj = target.default_event_space_bijector.forward_log_det_jacobian(x)
   return target.unnormalized_log_prob(y) + fldj

offset = 2
def bn_initialize(shape, key = random.PRNGKey(3727709)):
  return 3 * random.normal(key, shape + (num_dimensions,)) + offset

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/internal/backend/jax/ops.py:285: UserWarning: Explicitly requested dtype float64 requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return arr.astype(dtype)


In [6]:
num_chains = 512
K = 4
num_warmup, num_sampling = 100, 100000
total_samples = num_warmup + num_sampling

kernel = tfp.mcmc.HamiltonianMonteCarlo(target_log_prob_fn, init_step_size, 1)
kernel = tfp.experimental.mcmc.GradientBasedTrajectoryLengthAdaptation(kernel, num_warmup)
kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(kernel, num_warmup,
                                                  target_accept_prob = 0.75,
                                                  reduce_fn = tfp.math.reduce_log_harmonic_mean_exp)

# Initializing each chain (in a super chain) at the same location
initial_state = bn_initialize((K,))
initial_state = np.repeat(initial_state, num_chains // K, axis = 0)

In [7]:
result = tfp.mcmc.sample_chain(total_samples, initial_state,
                               kernel = kernel, seed = random.PRNGKey(1954))

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/substrates/jax/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '


In [8]:
result

StatesAndTrace(
  all_states=Array([[[-3.0322418e+00, -4.1480255e-01],
            [-1.8663765e+00,  4.5954847e-01],
            [-1.2851200e+00,  5.6593370e-01],
            ...,
            [ 2.0307572e+00, -1.2639621e+00],
            [ 1.3315489e+00,  1.6020894e-02],
            [ 7.3248911e-01, -1.9930919e+00]],
    
           [[-3.0322418e+00, -4.1480255e-01],
            [-1.8663765e+00,  4.5954847e-01],
            [-1.2851200e+00,  5.6593370e-01],
            ...,
            [ 2.0307572e+00, -1.2639621e+00],
            [ 1.3315489e+00,  1.6020894e-02],
            [ 7.3248911e-01, -1.9930919e+00]],
    
           [[-3.0322418e+00, -4.1480255e-01],
            [-1.8663765e+00,  4.5954847e-01],
            [-1.2851200e+00,  5.6593370e-01],
            ...,
            [ 2.0307572e+00, -1.2639621e+00],
            [ 1.3315489e+00,  1.6020894e-02],
            [ 7.3248911e-01, -1.9930919e+00]],
    
           ...,
    
           [[ 1.0459573e+00, -1.8397841e+00],
           

In [11]:
import tensorflow as tf
tf.executing_eagerly()

True

In [4]:
from tensorflow.python.ops.linalg_ops import matrix_determinant
# Inquire about _axis_size as a function, and what it is meant to do

def _axis_size():
  return

# What is Pavel's example? How can this be rewritten without tf (maybe JAX)?
# What is the purpose of removing TF
def _reduce_variance(x, axis= None, biased = False, keepdims = False):
  with tf.name_scope('reduce_variance'):
    x = tf.convert_to_tesnor(x, name = 'x')
    mean = tf.reduce_mean(x, axis=axis, keepdims=True)
    biased_var = tf.reduce_mean(
        tf.math.squared_difference(x, mean), axis = axis, keepdims = keepdims)
    if biased:
      return biased_var
    n = _axis_size(x, axis)
    return (n / (n-1.))*biased_var

def nested_rhat(result_state, num_super_chain):
  used_samples = result_state.shape[0]
  num_sub_chains = result_state.shape[1]
  num_dimensions = result_state.shape[2]

  chain_states = result_state.reshape(used_samples, -1, num_sub_chains,
                                      num_dimensions)
  state = tf.convert_to_tensor(chain_states, name = 'state')
  mean_chain = tf.reduce_mean(state, axis = 0)
  mean_super_chain = tf.reduce_mean(state, axis = [0, 2])
  variance_chain = _reduce_variance(state, axis = 0, biased = False)
  variance_super_chain = _reduce_variance(state, axis = 0, biased = False) \
     + tf.reduce_mean(variance_chain, axis = -1)

  W = tf.reduce_mean(variance_super_chain, axis = 0)
  B = _reduce_variance(mean_super_chain, axis = 0, biased = False)

  return tf.sqrt((W+B) / W)